In [18]:
import torch 
import torch.nn as nn
import torch.optim as optim 
import torchvision 
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

In [20]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

Files already downloaded and verified


In [21]:
latent_dim=100
lr=0.0002
beta1=0.5 
beta2=0.999
num_epochs=10 

generator 

In [30]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128*8*8),
            nn.ReLU(True),
            nn.Unflatten(1, (128, 8, 8)),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(64,momentum=0.8),
            nn.ReLU(),
            nn.Conv2d(64,3,kernel_size=3,padding=1),
            nn.Tanh()
        )
        
    def forward(self, z):
        img = self.model(z)
        
        return img

discreminator 

In [27]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator,self).__init__()
        self.model=nn.Sequential(
            nn.Conv2d(3,32,kernel_size=3,stride=2,padding=1),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Dropout(0,.25),
            nn.Conv2d(32,64,kernel_size=3,stride=2,padding=1),
            nn.ZeroPad2d((0,1,0,1)),
            nn.BatchNorm2d(64,momentum=0.8),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Dropout(0.25),
            nn.Conv2d(64,128,kernel_size=3,stride=2,padding=1),
            nn.BatchNorm2d(128,momentum=0.8),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Dropout(0.25),
            nn.Flatten(),
            nn.Linear(256*5*5,1),
            nn.Sigmoid()
        )

    
    def forward(self,img):
        validity=self.model(img)
        return validity


In [28]:
generator=Generator().to(device)
discriminator=Discriminator().to(device)
adversarial_loss=nn.BCELoss()
optimizer_G=optim.Adam(generator.parameters(),lr=lr,betas=(beta1,beta2))
optimizer_D=optim.Adam(discriminator.parameters(),lr=lr,betas=(beta1,beta2))


training the gan 

In [16]:
for epoch in range(num_epochs):
    for i ,batch in enumerate(train_loader):
        real_images=batch[0].to_device(device)
        valid=torch.ones(real_images.size(0),1,device=device),
        fake=torch.zeros(real_images.size(0),1,device=device)
        real_images=real_images.to(device)
        optimizer_D.zero_grad()
        z=torch.randn(real_images.size(0),latent_dim,device=device)
        fake_images=generator(z)
        real_loss=adversarial_loss(discriminator(real_images),valid)
        fake_loss=adversarial_loss(discriminator(fake_images.detach()),fake)
        d_loss=(real_loss+fake_loss)/2
        d_loss.backward()
        optimizer_D.step()
        optimizer_G.zero_grad()
        gen_images=generator(z)
        g_loss=adversarial_loss(discriminator(gen_images),valid)
        g_loss.backward()
        optimizer_G.step()

        if (i+1)%100==0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")
        if (epoch+1)%10==0:
            with torch.no_grad():
                z=torch.randn(16,latent_dim,device=device)
                gen_images=generator(z)
                gen_images=gen_images.cpu()
                grid=torchvision.utils.make_grid(gen_images, nrow=4, normalize=True)
                plt.figure(figsize=(8,8))
                plt.imshow(np.transpose(grid,(1,2,0)))
                plt.axis('off')
                plt.show()
    


AttributeError: 'Tensor' object has no attribute 'to_device'

In [29]:
for epoch in range(num_epochs):
    for i, batch in enumerate(train_loader):
        real_images = batch[0].to(device)  # Fixed: to_device -> to, removed extra comma
        valid = torch.ones(real_images.size(0), 1, device=device)  # Fixed: removed trailing comma
        fake = torch.zeros(real_images.size(0), 1, device=device)
        
        # Removed duplicate: real_images = real_images.to(device)
        
        # Train Discriminator
        optimizer_D.zero_grad()
        z = torch.randn(real_images.size(0), latent_dim, device=device)
        fake_images = generator(z)
        real_loss = adversarial_loss(discriminator(real_images), valid)
        fake_loss = adversarial_loss(discriminator(fake_images.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()
        
        # Train Generator
        optimizer_G.zero_grad()
        gen_images = generator(z)
        g_loss = adversarial_loss(discriminator(gen_images), valid)
        g_loss.backward()
        optimizer_G.step()
        
        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")
    
    # Fixed: moved to epoch level (removed extra indentation)
    if (epoch + 1) % 10 == 0:
        with torch.no_grad():
            z = torch.randn(16, latent_dim, device=device)
            gen_images = generator(z)
            gen_images = gen_images.cpu()
            grid = torchvision.utils.make_grid(gen_images, nrow=4, normalize=True)
            plt.figure(figsize=(8, 8))
            plt.imshow(np.transpose(grid, (1, 2, 0)))
            plt.axis('off')
            plt.show()  # Fixed: removed trailing comma

RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x3200 and 6400x1)